In [22]:
import numpy as np
import xarray as xr
from datatree import DataTree


In [11]:
rng = np.random.default_rng()

rng.standard_normal((2, 3))


array([[ 1.31140111,  1.26858656, -0.56489482],
       [-0.76722099, -0.01160823,  1.1627118 ]])

In [14]:
data = xr.DataArray(
    rng.standard_normal((2, 3)), dims=("x", "y"), coords={"x": [10, 20]}
)
data


<xarray.DataArray (x: 2, y: 3)>
array([[ 0.03696415, -0.50208564,  0.57432414],
       [ 0.28093333, -0.66521948,  0.54160987]])
Coordinates:
  * x        (x) int32 10 20
Dimensions without coordinates: y

In [16]:
ds = xr.Dataset(
    {
        "foo": data,
        "bar": ("x", [1, 2]),
        "baz": np.pi,
    }
)
ds


<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 10 20
Dimensions without coordinates: y
Data variables:
    foo      (x, y) float64 0.03696 -0.5021 0.5743 0.2809 -0.6652 0.5416
    bar      (x) int32 1 2
    baz      float64 3.142

In [19]:
ds2 = ds.interp(coords={"x": [10, 12, 14, 16, 18, 20]})
ds2


<xarray.Dataset>
Dimensions:  (x: 6, y: 3)
Coordinates:
  * x        (x) int32 10 12 14 16 18 20
Dimensions without coordinates: y
Data variables:
    foo      (x, y) float64 0.03696 -0.5021 0.5743 ... 0.2809 -0.6652 0.5416
    bar      (x) float64 1.0 1.2 1.4 1.6 1.8 2.0
    baz      float64 3.142

In [21]:
ds3 = xr.Dataset(
    {
        "people": ["alice", "bob"],
        "heights": ("people", [1.57, 1.82]),
    },
    coords={"species": "human"},
)
ds3


<xarray.Dataset>
Dimensions:  (people: 2)
Coordinates:
  * people   (people) <U5 'alice' 'bob'
    species  <U5 'human'
Data variables:
    heights  (people) float64 1.57 1.82

In [23]:
dt = DataTree.from_dict({"simulation/coarse": ds, "simulation/fine": ds2, "/": ds3})
dt


DataTree('None', parent=None)
│   Dimensions:  (people: 2)
│   Coordinates:
│     * people   (people) <U5 'alice' 'bob'
│       species  <U5 'human'
│   Data variables:
│       heights  (people) float64 1.57 1.82
└── DataTree('simulation')
    ├── DataTree('coarse')
    │       Dimensions:  (x: 2, y: 3)
    │       Coordinates:
    │         * x        (x) int32 10 20
    │       Dimensions without coordinates: y
    │       Data variables:
    │           foo      (x, y) float64 0.03696 -0.5021 0.5743 0.2809 -0.6652 0.5416
    │           bar      (x) int32 1 2
    │           baz      float64 3.142
    └── DataTree('fine')
            Dimensions:  (x: 6, y: 3)
            Coordinates:
              * x        (x) int32 10 12 14 16 18 20
            Dimensions without coordinates: y
            Data variables:
                foo      (x, y) float64 0.03696 -0.5021 0.5743 ... 0.2809 -0.6652 0.5416
                bar      (x) float64 1.0 1.2 1.4 1.6 1.8 2.0
                baz      float64 3.142

In [24]:
dt["simulation/coarse/foo"]

<xarray.DataArray 'foo' (x: 2, y: 3)>
array([[ 0.03696415, -0.50208564,  0.57432414],
       [ 0.28093333, -0.66521948,  0.54160987]])
Coordinates:
  * x        (x) int32 10 20
Dimensions without coordinates: y

In [25]:
dt["simulation/coarse"].ds

<xarray.DatasetView>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int32 10 20
Dimensions without coordinates: y
Data variables:
    foo      (x, y) float64 0.03696 -0.5021 0.5743 0.2809 -0.6652 0.5416
    bar      (x) int32 1 2
    baz      float64 3.142

In [26]:
dt["simulation"].mean(dim="x")

DataTree('simulation', parent=None)
├── DataTree('coarse')
│       Dimensions:  (y: 3)
│       Dimensions without coordinates: y
│       Data variables:
│           foo      (y) float64 0.1589 -0.5837 0.558
│           bar      float64 1.5
│           baz      float64 3.142
└── DataTree('fine')
        Dimensions:  (y: 3)
        Dimensions without coordinates: y
        Data variables:
            foo      (y) float64 0.1589 -0.5837 0.558
            bar      float64 1.5
            baz      float64 3.142

: 